In [172]:
circuit = {'gates': [['TOFFOLI', [1, 0, 2]], ['TOFFOLI', [0, 1, 2]], ['CNOT', [0, 1]], ['NOT', [2]], ['NOT', [0]], ['NOT', [1]], ['NOT', [0]], ['TOFFOLI', [0, 2, 1]]], 'bits': 3}

In [173]:
inputs = [list(map(int, bin(i)[2:].zfill(3))) for i in range(8)]

In [174]:
def apply(gate, a, b, c):
    tmp = [a, b, c]
    tmp = [tmp[i] for i in gate[1]] + [None]*(3-len(gate[1]))
    a, b, c = tuple(tmp)
    match gate[0]:
        case "TOFFOLI":
            return (a, b, c ^ (a & b))
        case "CNOT":
            return (a, a^b)
        case "NOT":
            return (0 if a else 1,)

In [175]:
def getOutputs(circuit):
    outputs = []
    for b in inputs:
        bits = b.copy()
        for gate in circuit:
            res = apply(gate, *bits)
            j=0
            for i in gate[1]:
                bits[i] = res[j]
                j+=1
        outputs.append(bits)
    return outputs

In [176]:
templates = [
    ([['CNOT', [0, 1]], ['NOT', [0]], ['CNOT', [0, 1]]], [['NOT', [0]], ['NOT', [1]]]),
    ([['CNOT', [1, 2]], ['CNOT', [0, 1]], ['CNOT', [1, 2]]], [['CNOT', [0, 1]], ['CNOT', [0, 2]]]),
    ([['TOFFOLI', [0, 1, 2]], ['CNOT', [0, 1]], ['TOFFOLI', [0, 1, 2]]], [['CNOT', [0, 1]], ['CNOT', [0, 2]]]),
    ([['TOFFOLI', [0, 1, 2]], ['NOT', [0]], ['TOFFOLI', [0, 1, 2]]], [['NOT', [0]], ['CNOT', [1, 2]]]),
    ([['NOT', [1]], ['TOFFOLI', [0, 1, 2]], ['NOT', [1]]], [['TOFFOLI', [0, 1, 2]], ['CNOT', [0, 2]]]),
    ([['CNOT', [0, 1]], ['TOFFOLI', [0, 1, 2]], ['CNOT', [0, 1]]], [['TOFFOLI', [0, 1, 2]], ['CNOT', [0, 2]]]),
    ([['CNOT', [0, 1]], ['CNOT', [1, 2]], ['CNOT', [0, 1]]], [['CNOT', [1, 2]], ['CNOT', [0, 2]]]),
    ([['TOFFOLI', [0, 1, 2]], ['TOFFOLI', [0, 2, 1]], ['TOFFOLI', [0, 1, 2]]], [['CNOT', [1, 2]], ['TOFFOLI', [0, 2, 1]], ['CNOT', [1, 2]]])
]

templates4 = [
    ([["TOFFOLI", [0, 2, 1]], ["CNOT", [1, 2]], ["TOFFOLI", [0, 1, 2]], ["TOFFOLI", [0, 2, 1]]], [["CNOT", [1, 2]], ["TOFFOLI", [0, 1, 2]]])
]

In [177]:
def optimize(a, b):
    if getOutputs([a, b]) == inputs:
        return []
    else:
        return [a, b]
    
def optimize3(a, b, c):
    if getOutputs([a, b, c]) == inputs:
        return []
    elif getOutputs([a, b, c]) == getOutputs([a, b]):
        return [a, b]
    elif getOutputs([a, b, c]) == getOutputs([b, c]):
        return [b, c]
    elif getOutputs([a, b, c]) == getOutputs([a, c]):
        return [a, c]
    elif getOutputs([a, b, c]) == getOutputs([b, a]):
        return [b, a]
    elif getOutputs([a, b, c]) == getOutputs([c, b]):
        return [c, b]
    elif getOutputs([a, b, c]) == getOutputs([c, a]):
        return [c, a]
    elif getOutputs([a, b, c]) == getOutputs([a]):
        return [a]
    elif getOutputs([a, b, c]) == getOutputs([b]):
        return [b]
    elif getOutputs([a, b, c]) == getOutputs([c]):
        return [c]
    else:
        for i, o in templates:
            if i[0][0] != a[0] or i[1][0] != b[0] or i[2][0] != c[0]:
                continue
            translate = {}
            tmp = a[1]
            if a[0] == "TOFFOLI":
                tmp = sorted(a[1][:2]) + a[1][2:]
            for j in range(len(a[1])):
                translate[tmp[j]] = i[0][1][j]
            ok=True
            for j in range(len(b[1])):
                if b[1][j] not in translate:
                    if i[1][1][j] in translate.values():
                        ok = False
                        break
                    translate[b[1][j]] = i[1][1][j]
                elif translate[b[1][j]] != i[1][1][j] and not(b[0] == "TOFFOLI" and j != 2 and translate[b[1][j]] in i[1][1][:2]):
                    ok = False
                    break
            if not ok:
                continue
            for j in range(len(c[1])):
                if c[1][j] not in translate:
                    if i[2][1][j] in translate.values():
                        ok = False
                        break
                    translate[c[1][j]] = i[2][1][j]
                elif translate[c[1][j]] != i[2][1][j] and not(c[0] == "TOFFOLI" and j != 2 and translate[c[1][j]] in i[2][1][:2]):
                    ok = False
                    break
            if ok:
                rev = {translate[e]: e for e in translate.keys()}
                return [[gate[0], [rev[e] for e in gate[1]]] for gate in o]
        return [a, b, c]
    
def optimize4(a, b, c, d):
    if getOutputs([a, b, c, d]) == inputs:
        return []
    elif getOutputs([a, b, c, d]) == getOutputs([a, b, c]):
        return [a, b, c]
    elif getOutputs([a, b, c, d]) == getOutputs([a, c, b]):
        return [a, c, b]
    elif getOutputs([a, b, c, d]) == getOutputs([a, b, d]):
        return [a, b, d]
    elif getOutputs([a, b, c, d]) == getOutputs([a, d, b]):
        return [a, d, b]
    elif getOutputs([a, b, c, d]) == getOutputs([b, a, c]):
        return [b, a, c]
    elif getOutputs([a, b, c, d]) == getOutputs([b, c, a]):
        return [b, c, a]
    elif getOutputs([a, b, c, d]) == getOutputs([b, a, d]):
        return [b, a, d]
    elif getOutputs([a, b, c, d]) == getOutputs([b, d, a]):
        return [b, d, a]
    elif getOutputs([a, b, c, d]) == getOutputs([c, a, b]):
        return [c, a, b]
    elif getOutputs([a, b, c, d]) == getOutputs([c, b, a]):
        return [c, b, a]
    elif getOutputs([a, b, c, d]) == getOutputs([c, a, d]):
        return [c, a, d]
    elif getOutputs([a, b, c, d]) == getOutputs([c, d, a]):
        return [c, d, a]
    elif getOutputs([a, b, c, d]) == getOutputs([a, b]):
        return [a, b]
    elif getOutputs([a, b, c, d]) == getOutputs([a, c]):
        return [a, c]
    elif getOutputs([a, b, c, d]) == getOutputs([a, d]):
        return [a, d]
    elif getOutputs([a, b, c, d]) == getOutputs([b, a]):
        return [b, a]
    elif getOutputs([a, b, c, d]) == getOutputs([b, c]):
        return [b, c]
    elif getOutputs([a, b, c, d]) == getOutputs([b, d]):
        return [b, d]
    elif getOutputs([a, b, c, d]) == getOutputs([c, a]):
        return [c, a]
    elif getOutputs([a, b, c, d]) == getOutputs([c, b]):
        return [c, b]
    elif getOutputs([a, b, c, d]) == getOutputs([c, d]):
        return [c, d]
    elif getOutputs([a, b, c, d]) == getOutputs([d, a]):
        return [d, a]
    elif getOutputs([a, b, c, d]) == getOutputs([d, b]):
        return [d, b]
    elif getOutputs([a, b, c, d]) == getOutputs([d, c]):
        return [d, c]
    elif getOutputs([a, b, c, d]) == getOutputs([a]):
        return [a]
    elif getOutputs([a, b, c, d]) == getOutputs([b]):
        return [b]
    elif getOutputs([a, b, c, d]) == getOutputs([c]):
        return [c]
    elif getOutputs([a, b, c, d]) == getOutputs([d]):
        return [d]
    else:
        for i, o in templates4:
            if i[0][0] != a[0] or i[1][0] != b[0] or i[2][0] != c[0] or i[3][0] != d[0]:
                continue
            translate = {}
            tmp = a[1]
            if a[0] == "TOFFOLI":
                tmp = sorted(a[1][:2]) + a[1][2:]
            for j in range(len(a[1])):
                translate[tmp[j]] = i[0][1][j]
            ok=True
            for j in range(len(b[1])):
                if b[1][j] not in translate:
                    if i[1][1][j] in translate.values():
                        ok = False
                        break
                    translate[b[1][j]] = i[1][1][j]
                elif translate[b[1][j]] != i[1][1][j] and not(b[0] == "TOFFOLI" and j != 2 and translate[b[1][j]] in i[1][1][:2]):
                    ok = False
                    break
            if not ok:
                continue
            for j in range(len(c[1])):
                if c[1][j] not in translate:
                    if i[2][1][j] in translate.values():
                        ok = False
                        break
                    translate[c[1][j]] = i[2][1][j]
                elif translate[c[1][j]] != i[2][1][j] and not(c[0] == "TOFFOLI" and j != 2 and translate[c[1][j]] in i[2][1][:2]):
                    ok = False
                    break
            if not ok:
                continue
            for j in range(len(d[1])):
                if d[1][j] not in translate:
                    if i[3][1][j] in translate.values():
                        ok = False
                        break
                    translate[d[1][j]] = i[3][1][j]
                elif translate[d[1][j]] != i[3][1][j] and not(d[0] == "TOFFOLI" and j != 2 and translate[d[1][j]] in i[3][1][:2]):
                    ok = False
                    break
            if ok:
                rev = {translate[e]: e for e in translate.keys()}
                return [[gate[0], [rev[e] for e in gate[1]]] for gate in o]
        return [a, b, c, d]

In [178]:
def simplify(circuit):
    tmp = circuit.copy()
    change = False
    for i in range(len(tmp)-1):
        if i >= len(tmp)-1:
            break
        opti = optimize(tmp[i], tmp[i+1])
        if len(opti) != 2:
            change = True
        tmp = tmp[:i] + opti + (tmp[i+2:] if i < len(tmp)-2 else [])
    for i in range(len(circuit)-2):
        if i >= len(tmp)-2:
            break
        opti = optimize3(tmp[i], tmp[i+1], tmp[i+2])
        if len(opti) != 3:
            change = True
        tmp = tmp[:i] + opti + (tmp[i+3:] if i < len(tmp)-3 else [])
    for i in range(len(circuit)-3):
        if i >= len(tmp)-3:
            break
        opti = optimize4(tmp[i], tmp[i+1], tmp[i+2], tmp[i+3])
        if len(opti) != 4:
            change = True
        tmp = tmp[:i] + opti + (tmp[i+4:] if i < len(tmp)-4 else [])
    for i in range(len(tmp)):
        if i >= len(tmp):
            break
        for j in range(len(tmp)):
            if j >= len(tmp):
                break
            if i != j:
                t = [tmp[k] for k in range(len(tmp)) if k != i and k != j]
                if getOutputs(tmp) == getOutputs(t):
                    change = True
                    tmp = t
    return simplify(tmp) if change else tmp

In [179]:
def printCircuit(circuit):
    print(str({"gates": circuit, "bits": 3}).replace("'", '"'))

In [180]:
import json
i = json.loads(input())
print(i)
s = simplify(i['gates'])
printCircuit(s)
print(getOutputs(i['gates']) == getOutputs(s))

{'gates': [['CNOT', [0, 2]], ['NOT', [1]], ['TOFFOLI', [0, 1, 2]], ['CNOT', [1, 0]], ['CNOT', [0, 2]], ['CNOT', [1, 2]], ['NOT', [0]], ['TOFFOLI', [2, 0, 1]]], 'bits': 3}
{"gates": [["CNOT", [1, 0]], ["NOT", [1]], ["TOFFOLI", [0, 1, 2]], ["TOFFOLI", [2, 0, 1]]], "bits": 3}
True
